<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
from github import Github, GithubException
import toml
import base64
from pprint import pprint
import re
from tabulate import tabulate
import numpy as np
import pandas as pd

## First exploration of github api in python

In [ ]:
#personal repo read token
token = '4fecdf4c37f5b641ca74d236582e70c72d282f68'
g=Github(base_url = 'https://github.dev.xero.com/api/v3',login_or_token=token)

In [ ]:
user = g.get_user("leo-browning")
repos = {repo.name:repo for repo in user.get_repos()}
repo_list = [[name,repo] for name, repo in repos.items()]

In [ ]:
repo_df =pd.DataFrame(repo_list,columns=['name','repo'])
repo_df

In [ ]:
repos['tokyo-docs-test'].get_dir_contents('.')[0]

In [ ]:
repos['tokyo-docs-test'].get_dir_contents('docs/source')

In [ ]:
base64.b64decode(repos['tokyo-docs-test'].get_contents('CODEOWNERS').content).decode("utf-8").splitlines()

In [ ]:
g=Github(base_url = 'https://github.dev.xero.com/api/v3')

In [ ]:
g.get_user('leo-browning')

## First exploration of toml load/dump

In [ ]:
meta = {"project meta":{'owner':'Leo Browning'}}

In [ ]:
toml.dumps(meta)

In [ ]:
tdoc = """
# This is a TOML document.
title = "TOML Example"
[owner]
name = "Tom Preston-Werner"
dob = 1979-05-27T07:32:00-08:00 # First class dates
"""
toml.loads(tdoc)

## CODOWNER checker

In [ ]:
codowner = """
# This is a comment.
# Each line is a file pattern followed by one or more owners.

# These owners will be the default owners for everything in
# the repo. Unless a later match takes precedence,
# @global-owner1 and @global-owner2 will be requested for
# review when someone opens a pull request.
*       @global-owner1 @global-owner2
# Order is important; the last matching pattern takes the most
# precedence. When someone opens a pull request that only
# modifies JS files, only @js-owner and not the global
# owner(s) will be requested for a review.
*.js    @js-owner
"""
ownerlines = codowner.split('\n')

In [ ]:
def is_global_owner_definition(line):
    """Checks that code ownership for all files in the repo is defined on the line passed"""
    return re.search('^\*\s', line)
def get_global_codeowners(repo):
    try:
        codeowner = repo.get_contents('CODEOWNERS')
    except GithubException:
        print(f'No owners defined for {repo.name}')
        return ''
    codeowner_lines = base64.b64decode(codeowner.content).decode('utf-8').splitlines()
    owners = []
    for line in codeowner_lines:
        if is_global_owner_definition(line):
            owners.append(line.strip('*').strip().split())

    assert len(owners) == 1, 'global owners multiply defined in CODEOWNERS'
    if owners:
        return ', '.join(owners[0])
    else:
        return ''
    
    

In [ ]:
get_global_codeowners('tokyo-docs-test')

## Repo lister

In [ ]:
def list_repos(username):
    user = g.get_user(username)
    repos = {repo.name:repo for repo in user.get_repos()}
    return repos

In [ ]:
list_repos('leo-browning')

In [ ]:
print(f'{"Repo":30} : {"Owners":30}')
print(''.center(63,'='))
for name, repo in list_repos('leo-browning').items():
    print(f'{name :30} : {get_global_codeowners(name)}')

In [ ]:
table = [[name,get_global_codeowners(name)] for name, repo in list_repos('leo-browning').items()]
headers = ["Repo", "Owners"]
print(tabulate(table, headers, tablefmt='rst'))

In [ ]:
def make_rst_table(repos):
    table = [[name,get_global_codeowners(name)] for name, repo in repos.items()]
    headers = ["Repo", "Owners"]
    return tabulate(table, headers, tablefmt='rst')

In [ ]:
print(make_rst_table(list_repos('leo-browning')))

In [ ]:
print(tabulate(pd.DataFrame([[1,2,3],[4,5,1]],columns=['a','b','c']).sort_values('c'),showindex=False,headers='keys',tablefmt='rst'))

In [ ]:
pd.DataFrame([[1,2,3],[4,5,1]],columns=['a','b','c'])

In [ ]:
repo_df.repo.apply(get_global_codeowners)

In [ ]:
repo_df['owner'] = repo_df.repo.map(get_global_codeowners)
table = tabulate(
            repo_df[['name', 'owner']].sort_values('owner'),
            showindex=False, headers='keys', tablefmt='rst'
        )


In [ ]:
print(table)